In [1]:
# Imports
import sys
from pathlib import Path

import cv2
from IPython.display import Image, clear_output, display
from picamera2 import Picamera2
from ultralytics import YOLO

In [2]:
# Find pyproject.toml in parent directories and add its directory to sys.path
path_curr = Path.cwd()
for parent in [path_curr] + list(path_curr.parents):
    pyproject_file = parent / "pyproject.toml"
    if pyproject_file.exists():
        sys.path.insert(0, str(parent))
        break

In [3]:
from src.camera.classes_camera import YOLOCameraManager
from src.servo.classes_servo import DualServoController

In [4]:
# initialize classes
dual_servo_controller = DualServoController()
yolo_camera_manager = YOLOCameraManager()

PCA9685 initialized successfully
PCA9685 initialized successfully
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


[0:05:35.767427780] [2494]  INFO Camera camera_manager.cpp:340 libcamera v0.6.0+rpt20251202
[0:05:35.826108730] [2653]  INFO IPAProxy ipa_proxy.cpp:180 Using tuning file /usr/share/libcamera/ipa/rpi/vc4/ov5647.json
[0:05:35.840943602] [2653]  INFO Camera camera_manager.cpp:223 Adding camera '/base/soc/i2c0mux/i2c@1/ov5647@36' for pipeline handler rpi/vc4
[0:05:35.841000899] [2653]  INFO RPI vc4.cpp:445 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media0 and ISP device /dev/media2
[0:05:35.841061289] [2653]  INFO RPI pipeline_base.cpp:1111 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[0:05:35.866412834] [2494]  INFO Camera camera.cpp:1215 configuring streams: (0) 640x480-XRGB8888/sRGB (1) 640x480-SGBRG10_CSI2P/RAW
[0:05:35.867342268] [2653]  INFO RPI vc4.cpp:620 Sensor: /base/soc/i2c0mux/i2c@1/ov5647@36 - Selected sensor format: 640x480-SGBRG10_1X10/RAW - Selected unicam format: 640x480-pGAA/RAW


In [5]:
yolo_camera_manager.display_results()

SyntaxError: '[31m[1mimgz[0m' is not a valid YOLO argument. Similar arguments are i.e. ['imgsz=640'].

    Arguments received: ['yolo', '--f=/home/maasyuki/.config/Code/User/globalStorage/ms-toolsai.jupyter/version-2025.9.1/jupyter/runtime/kernel-v3b9b28954306b70d649b6282a37ace8ca97a4aeb9.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of ['detect', 'segment', 'classify', 'obb', 'pose']
                MODE (required) is one of ['benchmark', 'val', 'predict', 'track', 'export', 'train']
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolo11n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmen